In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
#tf.enable_eager_execution()
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [60]:
nfl = pd.read_csv('data/nfl_scores.csv')

In [61]:
nfl['schedule_date'] = pd.to_datetime(nfl['schedule_date'])

In [62]:
nfl.index = nfl['schedule_date']

In [63]:
nfl = nfl.drop(columns='schedule_date')

In [64]:
nfl_1 = nfl['2014-09-04':'2019-12-15']

In [65]:
#normalize weather temp
nfl_1['weather_temperature'].max()
nfl_1['weather_temperature'] = pd.cut(nfl_1['weather_temperature'],
                                     bins = [-10,-5,0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,np.inf],
                                     labels = [-7.5,-2.5,2.5,7.5,12.5,17.5,22.5,27.5,32.5,37.5,42.5,
                                               47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5,92.5,97.5,102.5])

C:\Users\nickg\Miniconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
undecided = nfl_1

In [67]:
undecided_2 = undecided

In [68]:
undecided_2 = undecided_2.replace(['Arizona Cardinals', 
                                  'Atlanta Falcons',
                                  'Baltimore Ravens',
                                   'Buffalo Bills',
                                   'Carolina Panthers',
                                   'Chicago Bears',
                                   'Cincinnati Bengals',
                                   'Cleveland Browns',
                                   'Dallas Cowboys',
                                   'Denver Broncos',
                                   'Detroit Lions',
                                   'Green Bay Packers',
                                   'Houston Texans',
                                   'Indianapolis Colts',
                                   'Jacksonville Jaguars',
                                   'Kansas City Chiefs',
                                   'Los Angeles Chargers',
                                   'Los Angeles Rams',
                                   'Miami Dolphins',
                                   'Minnesota Vikings',
                                   'New Orleans Saints',
                                   'New York Giants',
                                   'New York Jets',
                                   'Oakland Raiders',
                                   'Philadelphia Eagles',
                                   'Pittsburgh Steelers',
                                   'San Francisco 49ers',
                                   'Seattle Seahawks',
                                   'Tampa Bay Buccaneers',
                                   'Tennessee Titans',
                                   'Washington Redskins',
                                  'St. Louis Rams',
                                  'San Diego Chargers',
                                  'New England Patriots',
                                  'Division',
                                  'Conference',
                                  'SuperBowl',
                                   'Superbowl',
                                  'WildCard'],
                                  [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                   18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                   32,33,34,35,36,37,37, 38])

In [69]:
#fav home
undecided_2 = undecided_2.replace(['ARI', 
                                  'ATL',
                                  'BAL',
                                   'BUF',
                                   'CAR',
                                   'CHI',
                                   'CIN',
                                   'CLE',
                                   'DAL',
                                   'DEN',
                                   'DET',
                                   'GB',
                                   'HOU',
                                   'IND',
                                   'JAX',
                                   'KC',
                                   'LAC',
                                   'LAR',
                                   'MIA',
                                   'MIN',
                                   'NO',
                                   'NYG',
                                   'NYJ',
                                   'OAK',
                                   'PHI',
                                   'PIT',
                                   'SF',
                                   'SEA',
                                   'TB',
                                   'TEN',
                                   'WAS',
                                  'STR',
                                  'SDC',
                                  'NE',
                                  ],
                                  [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                   18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                   32,33,34])

In [70]:
#fav home
undecided_2['home_fav'] = undecided_2['team_home'] == undecided_2['team_favorite_id']
undecided_2.home_fav = undecided_2.home_fav.astype(int)

In [71]:
features = ['schedule_week', 'weather_temperature', 'weather_wind_mph',
           'team_home', 'team_away', 'spread_favorite', 'over_under_line','home_fav'
            ]

In [72]:
x = undecided_2.loc[:, features]
x = x.replace('Wildcard', 18, regex=True)

In [73]:
conditions = [
    (undecided_2['home_fav'] == 0),
    (undecided_2['home_fav'] == 1)]
choices = [undecided_2['score_home']-undecided_2['spread_favorite'],
          undecided_2['score_home']+undecided_2['spread_favorite']]

undecided_2['score_home'] = np.select(conditions, choices, default='null')

In [74]:
undecided_2.score_home = undecided_2.score_home.astype(float)
undecided_2.score_away = undecided_2.score_away.astype(float)
undecided_2['home_win'] = undecided_2['score_home'] > undecided_2['score_away']


In [75]:
undecided_2.home_win = undecided_2.home_win.astype(int)

In [76]:
y = undecided_2.home_win

In [77]:
nfl_1['home_win'] = undecided_2['home_win']

C:\Users\nickg\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
#train data one day behind

In [79]:
train_data = x['2014-09-04':'2019-12-14']

In [80]:
train_y = undecided_2.home_win

In [81]:
train_y_data = train_y['2014-09-04':'2019-12-14']

In [82]:
#test range(optional)
test_data = x['2019-09-08':'2019-12-04']
test_y_data = train_y['2019-09-08':'2019-12-04']

In [83]:
#scikit DNN

In [84]:
from sklearn.neural_network import MLPClassifier

In [85]:
from sklearn.model_selection import train_test_split

In [86]:
model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=8*[94], random_state=10)

In [87]:
model.fit(train_data,train_y_data)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=[94, 94, 94, 94, 94, 94, 94, 94],
              learning_rate='constant', learning_rate_init=0.001, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [88]:
pred = model.predict(x)

In [89]:
pred = pd.DataFrame(pred)

In [90]:
nfl_1.tail(14)

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_win
schedule_date,,,,,,,,,,,,,,,,,
2019-12-15,2019,15,False,Arizona Cardinals,38.0,24.0,Cleveland Browns,CLE,-3.0,49,University of Phoenix Stadium,False,72.5,0.0,NaN,DOME,1
2019-12-15,2019,15,False,Carolina Panthers,24.0,30.0,Seattle Seahawks,SEA,-6.5,48.5,Bank of America Stadium,False,47.5,5.0,NaN,NaN,1
2019-12-15,2019,15,False,Cincinnati Bengals,13.0,34.0,New England Patriots,NE,-10.0,41.5,Paul Brown Stadium,False,32.5,4.0,NaN,NaN,0
2019-12-15,2019,15,False,Dallas Cowboys,44.0,21.0,Los Angeles Rams,LAR,-1.0,48.5,AT&T Stadium,False,72.5,7.0,NaN,DOME,1
2019-12-15,2019,15,False,Detroit Lions,17.0,38.0,Tampa Bay Buccaneers,TB,-4.5,45.5,Ford Field,False,72.5,0.0,NaN,DOME,0
2019-12-15,2019,15,False,Green Bay Packers,21.0,13.0,Chicago Bears,GB,-4.5,40.5,Lambeau Field,False,12.5,7.0,NaN,NaN,1
2019-12-15,2019,15,False,Kansas City Chiefs,23.0,3.0,Denver Broncos,KC,-10.0,45.5,Arrowhead Stadium,False,27.5,11.0,NaN,NaN,1
2019-12-15,2019,15,False,Los Angeles Chargers,10.0,39.0,Minnesota Vikings,MIN,-1.5,45.5,StubHub Center,False,62.5,11.0,NaN,NaN,0
2019-12-15,2019,15,False,New York Giants,36.0,20.0,Miami Dolphins,NYG,-3.0,46.5,MetLife Stadium,False,42.5,15.0,NaN,NaN,1


In [69]:
#accuarcy predictor day of
pred_final = pred.tail(14)
y = nfl_1['home_win']
y_final = y.tail(14)
y_final = pd.DataFrame(y_final)
accuracy_score(pred_final, y_final)

0.2857142857142857

In [ ]:
#TTS 3.0 final predictions
pred_final

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(90,110):
    model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=8*[depth], random_state=10)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
accuracy_score(pred,y)

In [ ]:
#decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(max_depth=28,random_state=50)

In [ ]:
model.fit(train_data, train_y_data)

In [ ]:
pred = model.predict(x)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
#decision tree predictions
pred.tail(14)

In [ ]:
final_pred = pred.tail(11)
y = nfl_1['home_win']
y_final = y.tail(11)
y_final = pd.DataFrame(y_final)

In [ ]:
accuracy_score(final_pred,y_final)

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(10,100,10):
    model = DecisionTreeClassifier(max_depth=depth, random_state = 50)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
#scalar data

In [ ]:
#from sklearn import preprocessing

In [ ]:
#scaler = preprocessing.MinMaxScaler()

In [ ]:
#train_data = scaler.fit_transform(train_data)

In [ ]:
#train_data = pd.DataFrame(train_data)

In [ ]:
#train_data

In [ ]:
#pred

In [ ]:
#pred[0].value_counts()

In [ ]:
#SGDClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
model = SGDClassifier(loss="hinge", penalty="l2", max_iter=100, random_state=42)

In [ ]:
model.fit(train_data, train_y_data)

In [ ]:
pred = model.predict(x)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
pred.tail(12)

In [ ]:
final_pred = pred.tail(12)
y = nfl_1['home_win']
y_final = y.tail(12)
y_final = pd.DataFrame(y_final)

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(10,200,10):
    model = SGDClassifier(loss="hinge", penalty="l2", max_iter=depth)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
accuracy_score(final_pred, y_final)

In [93]:
#gradient boosting classifier

In [56]:
from sklearn.ensemble import GradientBoostingClassifier

In [64]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.3,
        max_depth=1, random_state=0, loss='deviance')

In [65]:
model.fit(train_data, train_y_data)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [66]:
pred = model.predict(x)
pred = pd.DataFrame(pred)

In [67]:
pred.tail(14)

,0
1544,0
1545,0
1546,1
1547,0
1548,0
1549,0
1550,0
1551,0
1552,0
1553,0


In [68]:
accuracy_score(pred,y)

0.993581514762516

In [62]:
nfl_1.tail(14)

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_win
schedule_date,,,,,,,,,,,,,,,,,
2019-12-15,2019,15,False,Arizona Cardinals,38.0,24.0,Cleveland Browns,CLE,-3.0,49,University of Phoenix Stadium,False,72.5,0.0,NaN,DOME,1
2019-12-15,2019,15,False,Carolina Panthers,24.0,30.0,Seattle Seahawks,SEA,-6.5,48.5,Bank of America Stadium,False,47.5,5.0,NaN,NaN,1
2019-12-15,2019,15,False,Cincinnati Bengals,13.0,34.0,New England Patriots,NE,-10.0,41.5,Paul Brown Stadium,False,32.5,4.0,NaN,NaN,0
2019-12-15,2019,15,False,Dallas Cowboys,44.0,21.0,Los Angeles Rams,LAR,-1.0,48.5,AT&T Stadium,False,72.5,7.0,NaN,DOME,1
2019-12-15,2019,15,False,Detroit Lions,17.0,38.0,Tampa Bay Buccaneers,TB,-4.5,45.5,Ford Field,False,72.5,0.0,NaN,DOME,0
2019-12-15,2019,15,False,Green Bay Packers,21.0,13.0,Chicago Bears,GB,-4.5,40.5,Lambeau Field,False,12.5,7.0,NaN,NaN,1
2019-12-15,2019,15,False,Kansas City Chiefs,23.0,3.0,Denver Broncos,KC,-10.0,45.5,Arrowhead Stadium,False,27.5,11.0,NaN,NaN,1
2019-12-15,2019,15,False,Los Angeles Chargers,10.0,39.0,Minnesota Vikings,MIN,-1.5,45.5,StubHub Center,False,62.5,11.0,NaN,NaN,0
2019-12-15,2019,15,False,New York Giants,36.0,20.0,Miami Dolphins,NYG,-3.0,46.5,MetLife Stadium,False,42.5,15.0,NaN,NaN,1


In [32]:
#random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
model = RandomForestClassifier(n_estimators=100, max_depth=1,
                               random_state=0)

In [ ]:
model.fit(train_data, train_y_data)

In [ ]:
pred = model.predict(x)
pred = pd.DataFrame(pred)

In [ ]:
accuracy_score(pred,y)

In [ ]:
stop
print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(1,15):
    model = RandomForestClassifier(n_estimators=100, max_depth=depth,
                               random_state=0)
    model.fit(train_data,train_y_data)
    acc = accuracy_score(test_y_data, model.predict(test_data))
    print('{:>8d} | {:8.2f}'.format(depth, acc))

In [ ]:
pred.tail(14)

In [ ]:
nfl_1['home_win'].value_counts()

In [ ]:
x.tail(12)

In [ ]:
undecided_2.tail(14)

In [ ]:
#accuracy tests

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
precision_score(pred, y)

In [ ]:
recall_score(pred,y)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(pred,y)

In [ ]:
from sklearn.model_selection import cross_val_predict
y_scores = cross_val_predict(model, train_data, train_y_data, cv=3,
                            method='decision_function')

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, threshold = roc_curve(pred, y)
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1], [0,1], 'k--')
    plt.axis([0,1,0,1])
    plt.xlabel('flase positive rate')
    plt.ylabel('tru positive rate')

In [ ]:
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y,pred)

In [ ]:
DNN = (MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=8*[94], random_state=10))
DT = (DecisionTreeClassifier(max_depth=28,random_state=50))
SGD = (SGDClassifier(loss="hinge", penalty="l2", max_iter=100, random_state=42))
GBC = (GradientBoostingClassifier(n_estimators=100, learning_rate=.3,
        max_depth=6, random_state=0, loss='deviance'))
RF = (RandomForestClassifier(n_estimators=100, max_depth=1,
                               random_state=0))

In [ ]:
models=(DNN,DT,SGD,GBC,RF)
names = ('1','2','3','4','5')

In [ ]:
print('{:>8s} | {:8s}'.format('strategy', 'accuracy'))
print(20 * '-')
for mod in models:
    mod.fit(train_data, train_y_data)
    pred = mod.predict(x)
    pred = pd.DataFrame(pred)
    pred_final = pred.tail(12)
    y = nfl_1['home_win']
    y_final = y.tail(12)
    y_final = pd.DataFrame(y_final)
    acc = accuracy_score(pred_final, y_final)
    print(f"'model' | {acc}")

In [82]:

print('{:>8s} | {:8s}'.format('depth', 'accuracy'))
print(20 * '-')
for depth in range(1):
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.3,
        max_depth=1, random_state=0, loss='deviance')
    model.fit(train_data,train_y_data)
    pred = model.predict(x)
    pred = pd.DataFrame(pred)
    pred_final = pred.tail(14)
    y = nfl_1['home_win']
    y_final = y.tail(14)
    y_final = pd.DataFrame(y_final)
    acc = accuracy_score(pred_final, y_final)
    print('{:>8d} | {:8.2f}'.format(depth, acc))

   depth | accuracy
--------------------
       0 |     0.57
